In [2]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [3]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director

# Director Embedding
director_df의 결측행, 중복행을 해결하기 ---> director의 행길이 : 6807

### 결측행 추가
director_df에 없는 영화정보를 director를 'nm0000000'으로해서 뒤에 추가

In [4]:
no_dir_item_list = list(set(main_df['item']) - set(director_df['item']))

In [5]:
no_dir_item_df = pd.DataFrame([x for x in zip(no_dir_item_list, ['nm0000000']*len(no_dir_item_list))])

In [6]:
no_dir_item_df.columns=director_df.columns # 컬럼명 동일하게

In [7]:
director_df = pd.concat([director_df, no_dir_item_df]) # 기존 director_df 뒤에 감독없는 영화 추가

In [8]:
director_df

,item,director
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005
...,...,...
1299,73681,nm0000000
1300,32728,nm0000000
1301,32743,nm0000000
1302,106491,nm0000000


In [13]:
merged_direct_df = main_df.merge(director_df, how='left')

In [14]:
merged_direct_df

,user,item,time,director
0,11,4643,1230782529,nm0000318
1,11,170,1230782534,nm0812200
2,11,531,1230782539,nm0002140
3,11,616,1230782542,nm0718627
4,11,2140,1230782563,nm0000568
...,...,...,...,...
5708948,138493,44022,1260209449,nm0757858
5708949,138493,4958,1260209482,nm0601382
5708950,138493,68319,1260209720,nm0004303
5708951,138493,40819,1260209726,nm0003506


In [15]:
merged_direct_df.isna().sum()

user        0
item        0
time        0
director    0
dtype: int64

### 아이템 별 감독정보 통계량으로 나타내기
한 아이템당 여러 감독이 참여했을 때, 감독들의 평균(interaction/movie_cnt) 평점중 가장 큰 수

In [16]:
# 아래 코드 이해용
item_list = director_df[director_df['director'] == 'nm0000005']['item'].tolist()

In [17]:
# 아래 코드 이해용
item_list

[1237,
 5147,
 7327,
 2068,
 7396,
 7820,
 4422,
 7939,
 7938,
 7941,
 7937,
 2131,
 34155,
 7936,
 7940,
 7577,
 7942,
 8405,
 7328]

In [313]:
# 아래 코드 이해용
len(merged_direct_df[merged_direct_df['item'].isin(item_list)])

6360

In [147]:
# #감독별 받은 총 interaction 수 정보
# director_list = merged_direct_df['director'].unique()
# director_dict = {}
# for name in director_list:
#     dir_item_list = director_df[director_df['director'] == name]['item'].tolist()   #해당 감독이 기여한 영화 리스트
#     director_dict[name] = len(merged_direct_df[merged_direct_df['item'].isin(dir_item_list)])           #해당 감독이 기여한 영화들의 총 interaction 수

In [18]:
#감독별 받은 총 interaction 수 정보 / 감독이 기여한 영화수
director_list = merged_direct_df['director'].unique()
director_dict = {}
for name in director_list:
    dir_item_list = director_df[director_df['director'] == name]['item'].tolist()   #해당 감독이 기여한 영화 리스트
    director_dict[name] = len(merged_direct_df[merged_direct_df['item'].isin(dir_item_list)]) / (len(dir_item_list)+1)      #해당 감독이 기여한 영화들의 총 interaction 수

In [19]:
director_dict

{'nm0000318': 2859.4444444444443,
 'nm0812200': 583.7142857142857,
 'nm0002140': 430.3333333333333,
 'nm0718627': 1966.076923076923,
 'nm0000568': 824.3076923076923,
 'nm0001345': 1399.75,
 'nm0001317': 511.07142857142856,
 'nm0000186': 1423.8181818181818,
 'nm0922346': 675.5714285714286,
 'nm0001675': 3917.133333333333,
 'nm0001382': 173.66666666666666,
 'nm0000000': 230.99233716475095,
 'nm0000343': 797.0,
 'nm0730422': 224.33333333333334,
 'nm0432627': 952.0,
 'nm0000386': 1700.5454545454545,
 'nm0205157': 280.6666666666667,
 'nm0378893': 636.0,
 'nm0000416': 4461.615384615385,
 'nm0145336': 88.0,
 'nm0935095': 235.83333333333334,
 'nm0405632': 330.6666666666667,
 'nm0281945': 438.25,
 'nm0000965': 2549.0,
 'nm0294379': 701.0,
 'nm0811583': 1948.7142857142858,
 'nm0499724': 611.5714285714286,
 'nm0031976': 1898.4,
 'nm0781842': 886.1111111111111,
 'nm0935863': 170.0909090909091,
 'nm0027271': 497.5,
 'nm0833779': 186.0,
 'nm0833780': 186.0,
 'nm0919363': 3031.3333333333335,
 'nm0919

In [20]:
director_df[director_df['item'] == 1237]

,item,director
0,1237,nm0000005


In [21]:
director_df['director_inter'] = director_df['director'].map(director_dict)

In [22]:
director_df

,item,director,director_inter
0,1237,nm0000005,318.000000
1,5147,nm0000005,318.000000
2,7327,nm0000005,318.000000
3,2068,nm0000005,318.000000
4,7396,nm0000005,318.000000
...,...,...,...
1299,73681,nm0000000,230.992337
1300,32728,nm0000000,230.992337
1301,32743,nm0000000,230.992337
1302,106491,nm0000000,230.992337


In [23]:
director_df[director_df['item']==1]

,item,director,director_inter
2374,1,nm0005124,7436.166667


In [24]:
df = director_df.groupby('item')['director_inter'].max() # 한 아이템당 여러 감독이 참여했을 때, 감독들의 평균(interaction/movie_cnt) 평점중 가장 큰 수

In [25]:
df

item
1         7436.166667
2         1105.000000
3          373.250000
4           67.000000
5          349.800000
             ...     
118700     230.992337
118900     316.000000
118997     774.166667
119141     316.000000
119145    1856.500000
Name: director_inter, Length: 6807, dtype: float64

In [26]:
director_df = director_df.merge(df, how='left', on='item')
director_df

,item,director,director_inter_x,director_inter_y
0,1237,nm0000005,318.000000,318.000000
1,5147,nm0000005,318.000000,318.000000
2,7327,nm0000005,318.000000,318.000000
3,2068,nm0000005,318.000000,318.000000
4,7396,nm0000005,318.000000,318.000000
...,...,...,...,...
7204,73681,nm0000000,230.992337,230.992337
7205,32728,nm0000000,230.992337,230.992337
7206,32743,nm0000000,230.992337,230.992337
7207,106491,nm0000000,230.992337,230.992337


In [27]:
#결과 이해용
director_df[director_df['item']==25764] #한 영화에 2명의 감독 기여한 case 확인

,item,director,director_inter_x,director_inter_y
65,25764,nm0000036,218.111111,218.111111
4995,25764,nm0781292,123.000000,218.111111


In [28]:
director_df['director_info'] = director_df['director_inter_y']

In [29]:
director_df.drop_duplicates(
        subset=["item"], keep="first", inplace=True)

In [30]:
director_df

,item,director,director_inter_x,director_inter_y,director_info
0,1237,nm0000005,318.000000,318.000000,318.000000
1,5147,nm0000005,318.000000,318.000000,318.000000
2,7327,nm0000005,318.000000,318.000000,318.000000
3,2068,nm0000005,318.000000,318.000000,318.000000
4,7396,nm0000005,318.000000,318.000000,318.000000
...,...,...,...,...,...
7204,73681,nm0000000,230.992337,230.992337,230.992337
7205,32728,nm0000000,230.992337,230.992337,230.992337
7206,32743,nm0000000,230.992337,230.992337,230.992337
7207,106491,nm0000000,230.992337,230.992337,230.992337


In [33]:
director_df[['item','director_info']].to_csv(
    "director_inter_cnt.csv", index=False
)

In [32]:
director_df[['item','director_info']]

,item,director_info
0,1237,318.000000
1,5147,318.000000
2,7327,318.000000
3,2068,318.000000
4,7396,318.000000
...,...,...
7204,73681,230.992337
7205,32728,230.992337
7206,32743,230.992337
7207,106491,230.992337
